Primer de tot, importem el text que volem utilitzar:

In [17]:
with open('english_lyrics.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [18]:
print(text[:1000])

[Chorus: Opera Steve & Cam'ron]
Killa Cam, Killa Cam, Cam
Killa Cam, Killa Cam
Killa Cam, Cam
Killa Cam, Killa Cam, Cam
Killa Killa Killa Cam
Killa Cam, Cam, Killa (Killa!)
Killa Cam, Killa Cam, Cam (Bases loaded)
Killa Cam, Killa Cam (Uh-huh)
Killa Cam, Cam (Santana on second, Jim on third)
Killa Cam, Killa Cam, Cam (I'm at bat)
Killa Killa Killa Cam
Killa Cam, Cam, Killa (I'm 'bout to hit this shit out the world)
Killa Cam (Ugh, Heatmakerz), Killa Cam, Cam
Killa Cam, Killa Cam
Killa Cam, Cam (Hahahaha)
Killa Cam, Killa Cam, Cam
Killa Killa Killa Cam
Killa Cam, Cam, Killa (We  make this shit clap)
Killa Cam, Killa Cam, Cam
Killa Cam, Killa Cam
Killa Cam, Cam
Killa Cam, Killa Cam, Cam
Killa Killa Killa Cam (Killa! Killa!)
Killa Cam, Cam, Killa
[Verse 1]
With the goons I spar, stay in tune with ma (What up?)
She like, "Damn, this the realest since 'Kumbaya'"
Bomaye, Killa Cam, my Lord (My Lord)
Still the man with the pan, scrilla, fam, on board
Now bitches, they want to neuter me, nigga

In [19]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(vocab_size)
print(''.join(chars))

152
	
 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_`abcdefghijklmnopqrstuvwxyz{|}~ ¡´¿ÀÁÉÓÜàáâäçèéêëíîïñòóôöúüýāćēğīōœśūʼʿе​‎–—‘’“”•…′″⁠﻿


Tokenització a nivell de caràcter, a cada caràcter si li assigna un número.

In [20]:
stoi = {ch:i for i, ch in enumerate(chars)}
iost = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([iost[i] for i in l])

t = encode("Hola mon")
l = decode(t)

print(t)
print(l)

[42, 81, 78, 67, 2, 79, 81, 80]
Hola mon


In [21]:
import torch
device = 'cuda'
print(torch.cuda.is_available())
data = torch.tensor(encode(text), dtype=torch.long) # Tensor amb totes les dades
print(data.shape)
print(data[:1000])

True
torch.Size([30131396])
tensor([61, 37, 74, 81, 84, 87, 85, 28,  2, 49, 82, 71, 84, 67,  2, 53, 86, 71,
        88, 71,  2,  8,  2, 37, 67, 79,  9, 84, 81, 80, 63,  1, 45, 75, 78, 78,
        67,  2, 37, 67, 79, 14,  2, 45, 75, 78, 78, 67,  2, 37, 67, 79, 14,  2,
        37, 67, 79,  1, 45, 75, 78, 78, 67,  2, 37, 67, 79, 14,  2, 45, 75, 78,
        78, 67,  2, 37, 67, 79,  1, 45, 75, 78, 78, 67,  2, 37, 67, 79, 14,  2,
        37, 67, 79,  1, 45, 75, 78, 78, 67,  2, 37, 67, 79, 14,  2, 45, 75, 78,
        78, 67,  2, 37, 67, 79, 14,  2, 37, 67, 79,  1, 45, 75, 78, 78, 67,  2,
        45, 75, 78, 78, 67,  2, 45, 75, 78, 78, 67,  2, 37, 67, 79,  1, 45, 75,
        78, 78, 67,  2, 37, 67, 79, 14,  2, 37, 67, 79, 14,  2, 45, 75, 78, 78,
        67,  2, 10, 45, 75, 78, 78, 67,  3, 11,  1, 45, 75, 78, 78, 67,  2, 37,
        67, 79, 14,  2, 45, 75, 78, 78, 67,  2, 37, 67, 79, 14,  2, 37, 67, 79,
         2, 10, 36, 67, 85, 71, 85,  2, 78, 81, 67, 70, 71, 70, 11,  1, 45, 75,
        78, 

El 90% de les dades seran utilitzades per l'entrenament, i la resta per la validació.

In [22]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]
print(len(train_data))

27118256


Funció per crear un batch de dades. Per cada token, guardem el seu token següent per així tenir l'etiqueta real i poder entrenar efectivament.

In [23]:
vocab_size = len(chars)  
block_size = 8 
batch_size = 8

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

xb, yb = get_batch('train')
print(xb)
print(yb)
print('------')

for i in range(batch_size):
    for j in range(block_size):
        context = xb[i][:j+1]
        target = yb[i][j]
        print(f"When the data is {context.tolist()} the target is {target}")

tensor([[14,  2, 73, 75, 84, 78, 85,  2],
        [ 2, 86, 74, 71,  2, 68, 67, 84],
        [78, 70, 80,  9, 86,  2, 85, 78],
        [67, 91, 71, 84,  1, 37, 67, 86],
        [91,  2, 78, 81, 88, 71,  2, 74],
        [74, 71,  2, 78, 71, 67, 70, 85],
        [ 1, 51, 87, 75, 69, 77, 71, 84],
        [67, 80,  2, 73, 81,  1, 40, 67]], device='cuda:0')
tensor([[ 2, 73, 75, 84, 78, 85,  2, 67],
        [86, 74, 71,  2, 68, 67, 84,  1],
        [70, 80,  9, 86,  2, 85, 78, 75],
        [91, 71, 84,  1, 37, 67, 86, 69],
        [ 2, 78, 81, 88, 71,  2, 74, 75],
        [71,  2, 78, 71, 67, 70, 85,  2],
        [51, 87, 75, 69, 77, 71, 84,  2],
        [80,  2, 73, 81,  1, 40, 67, 85]], device='cuda:0')
------
When the data is [14] the target is 2
When the data is [14, 2] the target is 73
When the data is [14, 2, 73] the target is 75
When the data is [14, 2, 73, 75] the target is 84
When the data is [14, 2, 73, 75, 84] the target is 78
When the data is [14, 2, 73, 75, 84, 78] the target is 

Demostració del funcionament d'un cap d'atenció:

In [24]:
import torch
import torch.nn as nn
from torch.nn import functional as F

B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
# Definim les matrius de key, query i value
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias = False)
value = nn.Linear(C, head_size, bias = False)
k = key(x)
q = query(x)

# Creem la matriu d'atenció
wei = q @ k.transpose(-2, -1)
# Màscara per a que un token no pugui atendré als tokens següents
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v # Resultat del cap d'atenció

out.shape

torch.Size([4, 8, 16])

Model amb multi-head atention:

In [25]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# Paràmetres dimensionals i d'entrenament
n_embd = 384 # Dimensió dels embeddings
batch_size = 32 # Número de files del batch
block_size = 256 # Número de columnes del batch
max_iters = 5000 # Iteracions d'entrenament
eval_interval = 500 # Intervals per mostrar la pèrdua
learning_rate = 3e-4 
n_head = 6 # Número de caps d'atenció
n_layer = 6 # Número de blocks d'atenció
dropout = 0.2 # Probabilitat de dropout

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias = False)
        self.value = nn.Linear(n_embd, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)

        wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)

        v = self.value(x)
        out = wei @ v
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)
    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out)) # Projecció dels resultats dels caps concatenats
        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    # Bloc total d'antenció
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [26]:
class MultiheadModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        token_embed = self.token_embedding_table(idx) #Embedding dels tokens
        position_embed = self.position_embedding_table(torch.arange(T, device=idx.device)) # Embedding de posicions
        x = token_embed + position_embed # Suma simple per embedding de posicions
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        if targets is None:
            return logits, None
        else:
            # Pèrdua per l'entrenament
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
            return logits, loss

    def generate(self, idx, max_new_tokens):
        # Inferència
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

model = MultiheadModel()
m = model.to(device)
xb, yb = get_batch('train')
print(xb.shape)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

idx = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(idx, max_new_tokens=300)[0].tolist()))


torch.Size([32, 256])
torch.Size([8192, 152])
tensor(5.2616, device='cuda:0', grad_fn=<NllLossBackward0>)
	+…ućçiğ ‎“%hdKrKv–Q- Yʼ.ä⁠kîü'm“b-ʼō^E¡8k–u	OwJ8 gīWS9p(ā7еGS|ñ4…mW’s&À﻿k’VćúT5Rīw)ê″4​IuS;&
5wE=1ZG%W*èôóе​O´ōáru}B!xiÁSäc),yn‘	[E…#ýI%òÓOa"TbI<+′<F%á{L R”éU”èz1hY8tsEœ7…HJ6t1`@•òÉ|	SWGē #O&èÀÀ#
ʼ~zkū>]6`	"lw)+k3R]3z+7g“’j~ZWGuXjrX3āūZX.﻿…œ_[4—ú﻿``īê2mō0v:aâW1'@zZq2A″nī2pjhîaë@r¿jänu<ēTć[~(_Kä@PljB


In [27]:
optimizer3 = torch.optim.AdamW(m.parameters(), lr=learning_rate) # Utilitzem adam per entrenar
torch.cuda.empty_cache()
m.train()
# Bucle d'entrenament
for steps in range(max_iters):
    xb, yb = get_batch('train')

    logits, loss = m(xb, yb)
    optimizer3.zero_grad(set_to_none = True)
    loss.backward()
    optimizer3.step()
    if steps % eval_interval == 0:
        torch.save(m.state_dict(), '/modelSongs.pth')
        torch.save(optimizer3.state_dict(), '/optimizerSongs.pth')
        print("Iteration", steps, "with loss", loss.item())

print(loss.item())

Iteration 0 with loss 5.262340545654297
Iteration 500 with loss 2.294692277908325
Iteration 1000 with loss 1.886987328529358
Iteration 1500 with loss 1.7036086320877075
Iteration 2000 with loss 1.6508870124816895
Iteration 2500 with loss 1.529481053352356
Iteration 3000 with loss 1.5251659154891968
Iteration 3500 with loss 1.5452085733413696
Iteration 4000 with loss 1.49520742893219
Iteration 4500 with loss 1.474732518196106
1.3986258506774902


In [33]:
m.eval()
idx = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(idx, max_new_tokens=1000)[0].tolist()))

	: I'm a Levil DM Steel Radi: W—The Lands Lab Rands Rands Erah and Summer
Escept, I'm in verse, baby whire's when we blessin like that's Meral
Better flood, I'm blessin in this blues, c’mon
School, spit it-off means, baby law on, you wreck again
Yamma killer them hustler law muthafuckass, need again
Blues comin
If ya weather hang to the bent, ya weather nuthafuckas
I'm on two hand, hater them hands
We stop high to tink you and every probate up you think I'm baladed
On two game, ruggles he got discupped
Every fact is what the eagle, here never ever struggled, brother
Game, here n'm Canhole niggas that than it again
He's going bo7 at herback?
Cause I got her fact like the moms
Ruggling suffick, every later her speciably crazy
He's to the game, but I got her faced in the tings space
Tech man, except the song at the back and blown at her face
Put 'Early world and Either face cause I make a smace
Her attect off my shate, face of back rock him her in a body that time
You ktongue, instead and

Carregar un model guardat en disc dur:

In [32]:

modelLoaded = MultiheadModel()
optimizerLoaded = torch.optim.AdamW(modelLoaded.parameters(), lr=learning_rate)

network_state_dict = torch.load('/modelSongs.pth')
modelLoaded.load_state_dict(network_state_dict)

optimizer_state_dict = torch.load('/optimizerSongs.pth')
optimizerLoaded.load_state_dict(optimizer_state_dict)

modelLoaded.eval()
idx = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(idx, max_new_tokens=1000)[0].tolist()))

	e ten of my enboats, not somethin was succeet with a steppare
Elboats, feeling the richer flames are weed
Have been was getting drawness to the Cuberratures
Your fent waters to each to the waters
Housand the drocate that you laters
My tagers looking in the drama ringle I throw the East
I got good when even every daying to find the grind and get house
Wait in my threat while it last in the same ething that shit foul
With the jailified Bingle
Straighteen went in the light sent off the light of influent
Young Pan, I'm be up to get high, ten- six
Can't try the career, bitch! We the lighteen thing to tell 'em
Handle not something about sick upback - who come
Please not hanger, boughter, but make not something for the smile
Who could forget better because the Punk name to sit
Get a distance on the lesses, babe, byt soldier, bottle
Hanned on the good tips come with
Army plan niggas unleave speeds and go right was signer, put her and my eyes
And count made me to facture, come where your sneak